In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set Working Directory
import os
BASE_DIR = '/content/drive/MyDrive/speech_understanding_project'
DATA_DIR = os.path.join(BASE_DIR, 'data')
os.makedirs(DATA_DIR, exist_ok=True)


Mounted at /content/drive


In [2]:
!pip install transformers datasets torchaudio librosa soundfile tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 📍 Step 3: Install Dependencies

import torch
import torchaudio
import librosa
from tqdm import tqdm
import numpy as np
import soundfile as sf


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio
import torch

# Correct model name
model_id = "facebook/wav2vec2-xlsr-53-espeak-cv-ft"

# Load components separately
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_id)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_id)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_id)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2PhonemeCTCTokenizer'. 
The class this function is called from is 'Wav2Vec2CTCTokenizer'.


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [6]:
# Step 5: Define Feature Extraction Function
def extract_features_from_audio(file_path, model, processor):
    y, sr = librosa.load(file_path, sr=16000)
    inputs = processor(y, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(input_values=inputs.input_values.to(device))
    return outputs.last_hidden_state.squeeze(0).cpu().numpy()  # shape: [T, D]


In [7]:
# Step 6: Extract and Save Features for HRL & LRL
def extract_and_save_features_from_manifest(manifest_path, output_dir, prefix):
    os.makedirs(output_dir, exist_ok=True)
    with open(manifest_path, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(tqdm(lines)):
        path, dur = line.strip().split('\t')
        try:
            features = extract_features_from_audio(path, model, processor)
            fname = f"{prefix}_{i:03d}.npy"
            np.save(os.path.join(output_dir, fname), features)
        except Exception as e:
            print(f"Failed to process {path}: {e}")


In [8]:
#  Feature Extraction
hrl_manifest = os.path.join(DATA_DIR, 'hrl_manifest.tsv')
lrl_manifest = os.path.join(DATA_DIR, 'lrl_manifest.tsv')

hrl_feat_dir = os.path.join(DATA_DIR, 'features_hrl')
lrl_feat_dir = os.path.join(DATA_DIR, 'features_lrl')

extract_and_save_features_from_manifest(hrl_manifest, hrl_feat_dir, "hrl")
extract_and_save_features_from_manifest(lrl_manifest, lrl_feat_dir, "lrl")


  0%|          | 0/50 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

  2%|▏         | 1/50 [00:28<23:27, 28.72s/it]

Failed to process /content/drive/MyDrive/speech_understanding_project/data/commonvoice_hrl/hrl_000.wav: 'CausalLMOutput' object has no attribute 'last_hidden_state'


  4%|▍         | 2/50 [00:32<11:10, 13.98s/it]

...

Failed to process /content/drive/MyDrive/speech_understanding_project/data/commonvoice_lrl/lrl_047.wav: 'CausalLMOutput' object has no attribute 'last_hidden_state'


 98%|█████████▊| 49/50 [04:19<00:04,  4.08s/it]

Failed to process /content/drive/MyDrive/speech_understanding_project/data/commonvoice_lrl/lrl_048.wav: 'CausalLMOutput' object has no attribute 'last_hidden_state'


100%|██████████| 50/50 [04:26<00:00,  5.32s/it]

Failed to process /content/drive/MyDrive/speech_understanding_project/data/commonvoice_lrl/lrl_049.wav: 'CausalLMOutput' object has no attribute 'last_hidden_state'


In [9]:

print("Feature extraction complete!")



Feature extraction complete!
